In [1]:
import numpy as np
import copy

import heapq

from collections import defaultdict

from tqdm.notebook import tqdm

import matplotlib.pyplot as plt


In [2]:
with open('input_day_15.txt', 'r') as file:
    input_string = file.read()


test_input_string = """
########
#..O.O.#
##@.O..#
#...O..#
#.#.O..#
#...O..#
#......#
########

<^^>>>vv<v>>v<<
""".strip()

test_input_string = """
##########
#..O..O.O#
#......O.#
#.OO..O.O#
#..O@..O.#
#O#..O...#
#O..O..O.#
#.OO.O.OO#
#....O...#
##########

<vv>^<v^>v>^vv^v>v<>v^v<v<^vv<<<^><<><>>v<vvv<>^v^>^<<<><<v<<<v^vv^v>^
vvv<<^>^v^^><<>>><>^<<><^vv^^<>vvv<>><^^v>^>vv<>v<<<<v<^v>^<^^>>>^<v<v
><>vv>v^v^<>><>>>><^^>vv>v<^^^>>v^v^<^^>v^^>v^<^v>v<>>v^v^<v>v^^<^^vv<
<<v<^>>^^^^>>>v^<>vvv^><v<<<>^^^vv^<vvv>^>v<^^^^v<>^>vvvv><>>v^<<^^^^^
^><^><>>><>^^<<^^v>>><^<v>^<vv>>v>>>^v><>^v><<<<v>>v<v<v>vvv>^<><<>^><
^>><>^v<><^vvv<^^<><v<<<<<><^v<<<><<<^^<v<^^^><^>>^<v^><<<^>>^v<v^v<v^
>^>>^v>vv>^<<^v<>><<><<v<<v><>v<^vv<<<>^^v^>^^>>><<^v>>v^v><^^>>^<>vv^
<><^^>^^^<><vvvvv^v<v<<>^v<v>v<<^><<><<><<<^^<<<^<<>><<><^^^>^^<>^>v<>
^^>vv<^v^v<vv>^<><v<^v>^^^>>>^^vvv^>vvv<>>>^<^>>>>>^<<^v>^vvv<>^<><<v>
v^^>>><<^^<>>^v^<v^vv<>v^<<>^<^v^v><^<<<><<^<v><v<>vv>>v><v^<vv<>v^<<^
""".strip()

test_input_string = """
#######
#...#.#
#.....#
#..OO@#
#..O..#
#.....#
#######

<vv<<^^<<^^
""".strip()



In [3]:
grid, moves = input_string.split("\n\n")
grid = grid.split("\n")

grid = [list(r) for r in grid]

R = len(grid)
C = len(grid[0])

#for r in grid:
#    print("".join(r))
#print("")

for move in moves:

    #print("moving", move)

    if move == "\n":
        continue

    done_moving = False

    move_r, move_c = ((-1, 0), (1, 0), (0, 1), (0, -1))[list("^v><").index(move)]

    for r in range(R):
        for c in range(C):
            if not done_moving and grid[r][c] == "@":
                if grid[r+move_r][c+move_c] == "#":
                    done_moving = True
                    #print("directly hit wall")
                elif grid[r+move_r][c+move_c] == ".":
                    grid[r+move_r][c+move_c] = "@"
                    grid[r][c] = "."
                    done_moving = True
                    #print("directly moved")
                elif grid[r+move_r][c+move_c] == "O":
                    num_check = 2
                    while not done_moving:
                        if grid[r+move_r*num_check][c+move_c*num_check] == "#":
                            break
                        if grid[r+move_r*num_check][c+move_c*num_check] == ".":
                            grid[r][c] = "."
                            grid[r+move_r][c+move_c] = "@"
                            grid[r+move_r*num_check][c+move_c*num_check] = "O"
                            done_moving = True
                            #print("pushed boxes")
                        num_check += 1
                else:
                    pass
                    #print("can't push boxes")

    #print("")
    #for r in grid:
    #    print("".join(r))
    #print("")

tot = 0
for r in range(R):
    for c in range(C):
        if grid[r][c] == "O":
            tot += 100 * r + c
                            
tot

1446158

In [4]:
small_grid, moves = input_string.split("\n\n")
small_grid = small_grid.split("\n")

grid = []
for row in small_grid:
    new_row = ""
    for c in row:
        if c == ".":
            new_row += ".."
        if c == "#":
            new_row += "##"
        if c == "O":
            new_row += "[]"
        if c == "@":
            new_row += "@."
    grid.append(new_row)

grid = [list(r) for r in grid]

R = len(grid)
C = len(grid[0])


def check_move_box(pos_r, pos_c, move_r, move_c, grid):

    if grid[pos_r+move_r][pos_c+move_c] == "#" or grid[pos_r+move_r][pos_c+move_c+1] == "#":
        return False
    elif move_c == 0 and grid[pos_r+move_r][pos_c] == "." and grid[pos_r+move_r][pos_c+1] == ".": # moving both halves up or down
        return True
    elif move_c == 1 and grid[pos_r][pos_c+2] == ".": # moving both right
        return True
    elif move_c == -1 and grid[pos_r][pos_c-1] == ".": # moving both left
        return True
    else: # pushing another box
        if move_c == 0: # up or down
            aligned = grid[pos_r+move_r][pos_c+move_c] == "["
            misaligned_one_left = grid[pos_r+move_r][pos_c+move_c] == "]" and grid[pos_r+move_r][pos_c+move_c+1] == "."
            misaligned_one_right = grid[pos_r+move_r][pos_c+move_c] == "." and grid[pos_r+move_r][pos_c+move_c+1] == "["

            misaligned_two_boxes = grid[pos_r+move_r][pos_c+move_c] == "]" and grid[pos_r+move_r][pos_c+move_c+1] == "["

            if aligned or misaligned_one_left or misaligned_one_right:
                if aligned: 
                    new_pos_r = pos_r + move_r
                    new_pos_c = pos_c + move_c
                if misaligned_one_left:
                    new_pos_r = pos_r + move_r
                    new_pos_c = pos_c + move_c - 1
                if misaligned_one_right:
                    new_pos_r = pos_r + move_r
                    new_pos_c = pos_c + move_c + 1
                check = check_move_box(new_pos_r, new_pos_c, move_r, move_c, grid)
                return check
                
            if misaligned_two_boxes:
                success_1 = check_move_box(pos_r + move_r, pos_c + move_c - 1, move_r, move_c, grid)
                success_2 = check_move_box(pos_r + move_r, pos_c + move_c + 1, move_r, move_c, grid)
                return success_1 and success_2
                
        elif move_c == -1: # left
            check = check_move_box(pos_r, pos_c - 2, move_r, move_c, grid)
            return check
            
        elif move_c == 1: # right
            check = check_move_box(pos_r, pos_c+2, move_r, move_c, grid)
            return check


def recursive_move_box(pos_r, pos_c, move_r, move_c, grid):

    if (move_c != 1 and grid[pos_r+move_r][pos_c+move_c] in "[]") or (move_c != -1 and grid[pos_r+move_r][pos_c+move_c+1] in "[]"): # pushing another box

        if move_c == 0: # up or down
            aligned = grid[pos_r+move_r][pos_c+move_c] == "["
            misaligned_one_left = grid[pos_r+move_r][pos_c+move_c] == "]" and grid[pos_r+move_r][pos_c+move_c+1] == "."
            misaligned_one_right = grid[pos_r+move_r][pos_c+move_c] == "." and grid[pos_r+move_r][pos_c+move_c+1] == "["

            misaligned_two_boxes = grid[pos_r+move_r][pos_c+move_c] == "]" and grid[pos_r+move_r][pos_c+move_c+1] == "["

            if aligned or misaligned_one_left or misaligned_one_right:
                if aligned: 
                    new_pos_r = pos_r + move_r
                    new_pos_c = pos_c + move_c
                if misaligned_one_left:
                    new_pos_r = pos_r + move_r
                    new_pos_c = pos_c + move_c - 1
                if misaligned_one_right:
                    new_pos_r = pos_r + move_r
                    new_pos_c = pos_c + move_c + 1
                recursive_move_box(new_pos_r, new_pos_c, move_r, move_c, grid) # move other box first

            if misaligned_two_boxes:
                recursive_move_box(pos_r + move_r, pos_c + move_c - 1, move_r, move_c, grid)
                recursive_move_box(pos_r + move_r, pos_c + move_c + 1, move_r, move_c, grid)
                
        elif move_c == -1: # left
            recursive_move_box(pos_r, pos_c - 2, move_r, move_c, grid) # move other box first
            
        elif move_c == 1: # right
            recursive_move_box(pos_r, pos_c+2, move_r, move_c, grid) # move other box first

    grid[pos_r][pos_c] = "."
    grid[pos_r][pos_c + 1] = "."
    grid[pos_r+move_r][pos_c+move_c] = "["
    grid[pos_r+move_r][pos_c+move_c+1] = "]"

for move in moves:

    if move == "\n":
        continue

    move_r, move_c = ((-1, 0), (1, 0), (0, 1), (0, -1))[list("^v><").index(move)]

    done_moving = False
    didnt_get_stuck_boxes = True

    for r in range(R):
        for c in range(C):
            if not done_moving and grid[r][c] == "@":
                if grid[r+move_r][c+move_c] == "#":
                    done_moving = True
                elif grid[r+move_r][c+move_c] == ".":
                    grid[r+move_r][c+move_c] = "@"
                    grid[r][c] = "."
                    done_moving = True
                elif grid[r+move_r][c+move_c] == "[":
                    didnt_get_stuck_boxes =  check_move_box(r+move_r, c+move_c, move_r, move_c, grid)
                    if didnt_get_stuck_boxes:
                        recursive_move_box(r+move_r, c+move_c, move_r, move_c, grid)
                elif grid[r+move_r][c+move_c] == "]":
                    didnt_get_stuck_boxes = check_move_box(r+move_r, c+move_c-1, move_r, move_c, grid)
                    if didnt_get_stuck_boxes:
                        recursive_move_box(r+move_r, c+move_c-1, move_r, move_c, grid)
                
                if didnt_get_stuck_boxes and not done_moving:
                    grid[r+move_r][c+move_c] = "@"
                    grid[r][c] = "."
                    done_moving = True


tot = 0
for r in range(R):
    c = 0
    while True:
        if c >= C:
            break
        if grid[r][c] == "[":
            tot += 100 * r + c
            c += 1 # skip right half of box
        c += 1
                            
tot

1446175